In [7]:
import pandas as pd
from time import sleep


# Liste des fichiers à analyser
datafiles = ['250101_Liste_fonds_label_ISR-6.xlsx', '250101_Liste_fonds_label_ISR-7.xlsx']

# Liste des mots-clés à rechercher
keywords = ['inclusion', 'diversity', 'emploi', 'health', 'santé', 'equity']

# Initialiser une liste pour stocker les résultats
total_matching_cells = []

for datafile in datafiles:
    # Charger le fichier Excel
    df = pd.read_excel(datafile)
    
    # Sélectionner la deuxième colonne (B), peu importe son nom réel
    col_b_name = df.columns[1]  # La deuxième colonne a l'index 1
    
    # Filtrer les cellules contenant au moins un des mots-clés
    matching_cells = df[df[col_b_name].astype(str).str.contains('|'.join(keywords), case=False, na=False)][col_b_name].tolist()
    
    # Ajouter les résultats avec le nom du fichier comme référence
    total_matching_cells.extend([(datafile, cell) for cell in matching_cells])

# Convertir en DataFrame et sauvegarder le résultat
output_df = pd.DataFrame(total_matching_cells, columns=['Fichier', 'Nom du Fonds'])
output_file = 'matching_cells.xlsx'
output_df.to_excel(output_file, index=False)

print(f"Résultats enregistrés dans {output_file}")

# Recherche d'informations sur chaque fonds
fonds_info = []

for _, row in output_df.iterrows():
    fund_name = row['Nom du Fonds']
    query = f"{fund_name} documentation rendement volatilité"
    search_results = list(ddgs.text(query, max_results=1))
    
    # Simuler une pause pour éviter un blocage
    sleep(2)
    
    # Ajouter les résultats trouvés (extrait du premier résultat)
    if search_results:
        fonds_info.append([fund_name, search_results[0]['title'], search_results[0]['href']])
    else:
        fonds_info.append([fund_name, "Aucune information trouvée", ""])

# Sauvegarder les informations dans un fichier Excel
fonds_df = pd.DataFrame(fonds_info, columns=['Nom du Fonds', 'Titre de la Source', 'Lien'])
fonds_output_file = 'fonds_info.xlsx'
fonds_df.to_excel(fonds_output_file, index=False)

print(f"Informations des fonds enregistrées dans {fonds_output_file}")


ModuleNotFoundError: No module named 'duckduckgo_search'

In [ ]:
tickers = [
    # ==== OAT VERTES via ETF ====
    "EGOA.PA",     # Amundi Euro Government Green Bond UCITS ETF (Yahoo Finance)
    "GRNB",        # Lyxor Green Bond ETF - DR (Lyxor devenu Amundi) - ticker US
    "GSGB",        # Goldman Sachs Global Green Bond UCITS ETF (Yahoo Finance)
    "GRN0.DE",     # iShares € Green Bond UCITS ETF (Xetra - Allemagne)

    # ==== ETF OBLIGATAIRES INDEXÉS INFLATION ====
    "TIP",         # iShares TIPS Bond ETF (US Treasury Inflation-Protected Securities)

    # ==== ETF OBLIGATAIRES VERTS ====
    "BGRN",        # iShares Global Green Bond ETF (USD)

    # ==== ETF À IMPACT / THÉMATIQUES ESG ====
    "UBS6.DE",     # UBS Global Gender Equality ETF (Yahoo - Xetra)
    "GEND.PA",     # Amundi Global Gender Equality UCITS ETF (Yahoo - Paris)
    "AGED",        # iShares Ageing Population UCITS ETF (Yahoo - US)
    "VBUG.DE",     # VanEck Bionic Engineering UCITS ETF (Yahoo - Xetra)

    # ==== ACTIONS DIRECTES ====
    "SW.PA",       # Sodexo (France)
    "CAP.PA",      # Capgemini (France)
    "EL.PA",       # EssilorLuxottica (France) - nom correct sur Yahoo
    "2353.TW",     # Acer Inc. (Taiwan)
    "YAMCY",       # Yamaha Corp. (ADR - US)
    "7951.T",      # Yamaha Corp. (Tokyo)

    # ==== (OPTIONNEL) À VÉRIFIER ====
    # "GB00BYYT3J23.PAR",  # Amundi Green Bonds ETF - pas reconnu sur Yahoo à ce jour
]
